In [1]:
import pandas as pd
import pyodbc 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Import Datasets

In [2]:
conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=servername;'
                      'DATABASE=AdventureWorksDW2019;'
                     'Trusted_Connection=yes;')

sql_for_df1 = """SELECT  [BirthDate],
[MaritalStatus],
[YearlyIncome],
[Gender],[NumberChildrenAtHome], dg.PostalCode, 
dc.[TotalChildren],
[EnglishEducation],[EnglishOccupation],
[HouseOwnerFlag],[CustomerKey],
[NumberCarsOwned] 
FROM [dbo].[DimCustomer] as dc
Join [dbo].[DimGeography] as dg
on dc.[GeographyKey] = dg.GeographyKey
"""

sales = pd.read_sql(sql_for_df1, conn)

conn.close()

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
sales.head()

,BirthDate,MaritalStatus,YearlyIncome,Gender,NumberChildrenAtHome,PostalCode,TotalChildren,EnglishEducation,EnglishOccupation,HouseOwnerFlag,CustomerKey,NumberCarsOwned
0,1971-10-06,M,90000.0,M,0,4700,2,Bachelors,Professional,1,11000,0
1,1976-05-10,S,60000.0,M,3,3198,3,Bachelors,Professional,0,11001,1
2,1971-02-09,M,60000.0,M,3,7001,3,Bachelors,Professional,1,11002,1
3,1973-08-14,S,70000.0,F,0,2113,0,Bachelors,Professional,0,11003,1
4,1979-08-05,S,80000.0,F,5,2500,5,Bachelors,Professional,1,11004,4


In [4]:
conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=PRECIOUS\DONSQLEXPRESS;'
                      'DATABASE=AdventureWorks2019;'
                     'Trusted_Connection=yes;')

sql_for_df1 = """select 
PC.[Name] AS [Product Category],
PS.[Name] AS [Product Subcategory], p.Name, soh.SalesOrderID, sod.ProductID, soh.SubTotal,soh.customerID, 
sod.OrderQty, sod.UnitPrice, soh.TotalDue, st.Name as region, soh.freight, soh.TaxAmt,
sod.LineTotal, soh.OrderDate, soh.ShipDate from
Sales.SalesOrderDetail as sod
join Sales.SalesOrderHeader as soh 
on sod.SalesOrderID=soh.SalesOrderID
join Production.Product as p
on sod.ProductID=p.ProductID
INNER JOIN Sales.SalesTerritory AS St
ON Soh.[TerritoryID] = St.[TerritoryID]
LEFT JOIN [Production].[Product] PP ON PP.ProductID = Sod.ProductID
LEFT JOIN [Production].[ProductSubcategory] PS ON PS.[ProductSubcategoryID] = PP.ProductSubcategoryID
LEFT JOIN [Production].[ProductCategory] PC ON PC. [ProductCategoryID] = PS.ProductCategoryID
"""

data = pd.read_sql(sql_for_df1, conn)

conn.close()

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
data.head()

,Product Category,Product Subcategory,Name,SalesOrderID,ProductID,SubTotal,customerID,OrderQty,UnitPrice,TotalDue,region,freight,TaxAmt,LineTotal,OrderDate,ShipDate
0,Bikes,Mountain Bikes,"Mountain-100 Black, 42",43659,776,20565.6206,29825,1,2024.994,23153.2339,Southeast,616.0984,1971.5149,2024.994,2011-05-31,2011-06-07
1,Bikes,Mountain Bikes,"Mountain-100 Black, 44",43659,777,20565.6206,29825,3,2024.994,23153.2339,Southeast,616.0984,1971.5149,6074.982,2011-05-31,2011-06-07
2,Bikes,Mountain Bikes,"Mountain-100 Black, 48",43659,778,20565.6206,29825,1,2024.994,23153.2339,Southeast,616.0984,1971.5149,2024.994,2011-05-31,2011-06-07
3,Bikes,Mountain Bikes,"Mountain-100 Silver, 38",43659,771,20565.6206,29825,1,2039.994,23153.2339,Southeast,616.0984,1971.5149,2039.994,2011-05-31,2011-06-07
4,Bikes,Mountain Bikes,"Mountain-100 Silver, 42",43659,772,20565.6206,29825,1,2039.994,23153.2339,Southeast,616.0984,1971.5149,2039.994,2011-05-31,2011-06-07


In [6]:
#rename customerkey in first dataset to customerID for merging
sales.rename(columns={'CustomerKey':'customerID'},inplace=True)

In [7]:
df = pd.merge(sales,data,how='inner',on='customerID')

# User-based filtering recommendation engine

In [8]:
# Extract relevant columns
user_item_data = df[['customerID', 'Name', 'OrderQty']]

# Group data by user-item pairs and aggregate PurchaseFrequency
user_item_df = user_item_data.groupby(['customerID', 'Name']).sum().reset_index()

# Create user-item matrix
user_item_matrix = pd.pivot_table(user_item_df, values='OrderQty', index='customerID', columns='Name', fill_value=0)

In [9]:
user_item_matrix.head()

Name,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
customerID,,,,,,,,,,,,,,,,,,,,,
11000,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11001,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,2,0,0,0
11002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11003,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11004,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
user_based_cosine_matrix = cosine_similarity(user_item_matrix)

# Convert cosine similarity matrix to DataFrame for better readability
user_item_similarity_matrix = pd.DataFrame(user_based_cosine_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

In [11]:
user_item_similarity_matrix.head()

customerID,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
customerID,,,,,,,,,,,,,,,,,,,,,
11000,1.000000,0.196116,0.000000,0.235702,0.288675,0.000000,0.000000,0.250000,0.133631,0.000000,...,0.0,0.353553,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000
11001,0.196116,1.000000,0.000000,0.369800,0.226455,0.000000,0.372104,0.098058,0.314485,0.248069,...,0.0,0.000000,0.000000,0.160128,0.480384,0.0,0.620174,0.0,0.0,0.000000
11002,0.000000,0.000000,1.000000,0.166667,0.000000,0.000000,0.223607,0.176777,0.188982,0.223607,...,0.0,0.000000,0.500000,0.288675,0.000000,0.5,0.000000,0.0,0.5,0.000000
11003,0.235702,0.369800,0.166667,1.000000,0.000000,0.136083,0.447214,0.235702,0.503953,0.149071,...,0.0,0.000000,0.333333,0.192450,0.192450,0.0,0.298142,0.0,0.0,0.333333
11004,0.288675,0.226455,0.000000,0.000000,1.000000,0.333333,0.182574,0.288675,0.154303,0.182574,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.365148,0.0,0.0,0.000000


In [12]:
user_based_cosine_matrix.shape

(18484, 18484)

In [13]:
#Creating the function that brings out the top 5 most similar customers
def get_top_similar_customers(user_id, similarity_matrix, top_n=5):
    
    similarities = similarity_matrix[user_id]
    top_similar_customers = list(similarities.sort_values(ascending=False).items())
    top_similar_customers = [(cust_id, sim) for cust_id, sim in top_similar_customers if cust_id != user_id]
    similar_customer = [item[0] for item in top_similar_customers[:top_n]]
    return similar_customer 

In [14]:
get_top_similar_customers(11008, user_item_similarity_matrix)

[13089, 13909, 18442, 12861, 18126]

# Item based filtering recommendation engine

In [15]:
# Extract relevant columns
user_item_data = df[['customerID', 'Name', 'OrderQty']]

# Group data by user-item pairs and aggregate PurchaseFrequency
user_item_df = user_item_data.groupby(['customerID', 'Name']).sum().reset_index()

# Create user-item matrix
item_based_matrix = pd.pivot_table(user_item_df, values='OrderQty', index='Name', columns='customerID', fill_value=0)

In [16]:
item_based_matrix.head()

customerID,11000,11001,11002,11003,11004,11005,11006,11007,11008,11009,...,29474,29475,29476,29477,29478,29479,29480,29481,29482,29483
Name,,,,,,,,,,,,,,,,,,,,,
AWC Logo Cap,0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
All-Purpose Bike Stand,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bike Wash - Dissolver,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Classic Vest, L",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Classic Vest, M",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
cosine_item_matrix = cosine_similarity(item_based_matrix)

# Convert cosine similarity matrix to DataFrame for better readability
item_based_similarity_matrix = pd.DataFrame(cosine_item_matrix, index=item_based_matrix.index, columns=item_based_matrix.index)

In [18]:
item_based_similarity_matrix.head()

Name,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
Name,,,,,,,,,,,,,,,,,,,,,
AWC Logo Cap,1.000000,0.047528,0.076049,0.019319,0.026480,0.022415,0.133142,0.130525,0.086280,0.053509,...,0.020752,0.040526,0.021614,0.041935,0.024216,0.035218,0.258073,0.105654,0.102868,0.092838
All-Purpose Bike Stand,0.047528,1.000000,0.035198,0.008865,0.008776,0.023878,0.050810,0.076207,0.099431,0.029018,...,0.015475,0.016117,0.000000,0.008934,0.000000,0.017508,0.041854,0.009746,0.006598,0.021301
Bike Wash - Dissolver,0.076049,0.035198,1.000000,0.018098,0.024634,0.014624,0.094857,0.158112,0.146269,0.045912,...,0.007898,0.024677,0.008226,0.004560,0.000000,0.008935,0.175070,0.028188,0.023573,0.019931
"Classic Vest, L",0.019319,0.008865,0.018098,1.000000,0.010153,0.005525,0.078894,0.053666,0.040600,0.006714,...,0.008951,0.018646,0.000000,0.000000,0.010446,0.000000,0.074738,0.011276,0.011451,0.020536
"Classic Vest, M",0.026480,0.008776,0.024634,0.010153,1.000000,0.010938,0.108723,0.043637,0.062517,0.019940,...,0.008861,0.000000,0.009229,0.020464,0.010340,0.000000,0.077110,0.026045,0.018892,0.020329


In [21]:
#Function bringing out top 5 most common products
def item_based_recommendation(item_based_similarity_matrix, item_id, top_n=5):
    similar_items = item_based_similarity_matrix.loc[item_id].sort_values(ascending=False)
    similar_items = similar_items.drop(item_id)
    top_similar_items = similar_items.head(top_n)
    return top_similar_items.index.tolist()

In [22]:
item_id = 'AWC Logo Cap'
item_based_recommendations = item_based_recommendation(item_based_similarity_matrix, item_id)

print(f'itembased: {item_based_recommendations}')

itembased: ['Water Bottle - 30 oz.', 'Touring Tire Tube', 'Mountain Bottle Cage', 'Road Tire Tube', 'Sport-100 Helmet, Red']
